In [1]:
import pandas as pd

In [20]:
df = pd.read_csv('../data/source/df_joined.csv')
df['default'] = df['default'].astype('int')
df['obs_date'] = pd.to_datetime(df['obs_date'])
df['year'] = df['obs_date'].dt.year

In [21]:
def filter_df(df, type):
    # Sort the DataFrame by 'ID' and 'obs_date' to ensure observations are ordered correctly
    df.sort_values(by=['ID', 'obs_date'], inplace=True)

    # Find the first occurrence of default = 1 for each ID
    first_default_indices = df[df['default'] == 1].groupby('ID').head(1)

    # Merge the first default indices with the original DataFrame to get the corresponding cutoff dates
    df = df.merge(first_default_indices[['ID', 'obs_date']], on='ID', suffixes=('', '_cutoff'), how='left')

    # Rename the 'obs_date_cutoff' column to 'cutoff_date'
    df.rename(columns={'obs_date_cutoff': 'cutoff_date'}, inplace=True)

    # Forward fill the cutoff dates to fill the missing values
    df['cutoff_date'] = df.groupby('ID')['cutoff_date'].ffill()

    if type == 'filter':
        # Filter out rows where 'obs_date' is greater than 'cutoff_date' for each ID
        filtered_df = df[df['obs_date'] <= df['cutoff_date']]
    elif type == 'change':
        
        df['new_default'] = 0 
        df.loc[df['obs_date'] >= df['cutoff_date'], 'new_default'] = 1

    # Drop the 'cutoff_date' column if it's no longer needed
    # filtered_df.drop(columns=['cutoff_date'], inplace=True)

    return df

In [22]:
df_1 = filter_df(df, type='change')
df_1
# df_1 = df_1[['ID', 'obs_date', 'cutoff_date', 'default', 'new_default']]

,ID,obs_date,Var_01,Var_02,Var_03,Var_04,Var_05,Var_06,Var_07,Var_08,...,Var_34,Var_35,Var_36,Var_37,Var_38,Var_39,default,year,cutoff_date,new_default
0,1574,2015-12-31,1.459814e+05,2.084565e+06,121857.381239,1.340909,0.314156,6.185654e+02,0.766667,1.701055e+05,...,1.177748e+06,0.435015,5.981527e+05,3.503623,2.474262e+03,3.711392e+04,0,2015,NaT,0
1,1574,2016-12-31,2.430558e+05,2.667237e+06,128323.855871,1.775559,0.328272,4.116193e+02,0.760961,2.076913e+05,...,1.606237e+06,0.397790,6.814891e+05,3.274769,1.313236e+04,1.061660e+05,0,2016,NaT,0
2,1574,2017-12-31,1.133773e+05,2.470817e+06,95445.639441,1.081923,0.191255,1.513055e+03,1.000000,2.369280e+05,...,1.662793e+06,0.327027,4.466381e+05,1.873159,1.092604e+04,8.584896e+03,0,2017,NaT,0
3,1574,2018-12-31,1.970604e+05,3.136429e+06,156790.021348,2.051041,0.175084,3.074959e+03,0.730127,2.188989e+05,...,2.225452e+06,0.290450,4.384314e+05,1.975149,1.578360e+03,1.009822e+05,0,2018,NaT,0
4,1574,2019-12-31,1.766260e+05,2.413088e+06,154848.635273,1.096836,0.180196,2.177675e+03,0.652214,1.279507e+05,...,1.559727e+06,0.353639,4.785634e+05,3.677625,1.799459e+03,1.559375e+04,0,2019,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148924,99991694,2015-12-31,6.192680e+05,2.533073e+06,147127.072077,1.708610,-0.026182,9.833619e+03,0.064670,1.122458e+05,...,2.761724e+05,0.143083,-1.000410e+05,-0.819475,1.655752e+05,2.568284e+05,0,2015,NaT,0
148925,99991954,2018-12-31,4.354878e+07,5.113584e+07,800365.030111,2.846886,-2.546194,4.679316e+05,-6.612175,-1.673950e+07,...,-1.768798e+07,1.345902,5.351337e+07,-3.288766,3.360523e+06,2.825179e+07,0,2018,NaT,0
148926,99991954,2019-12-31,1.798610e+07,2.201727e+07,8853.989244,0.397986,-0.526652,1.008313e+06,-0.629559,-1.082530e+07,...,-2.317558e+07,2.052609,2.343495e+07,-2.387182,3.530658e+06,-2.720675e+07,0,2019,NaT,0
148927,99991954,2020-12-31,1.267978e+07,1.571147e+07,111018.408388,0.266830,-0.524300,1.040206e+06,-0.320276,-7.251975e+06,...,-3.180862e+07,3.024547,3.169654e+07,-5.102660,5.735074e+04,-3.484031e+07,0,2020,NaT,0


In [24]:
df_1.groupby('year')['new_default'].mean()

year
2015    0.155205
2016    0.183157
2017    0.192827
2018    0.211867
2019    0.232698
2020    0.256101
2021    0.370725
Name: new_default, dtype: float64